# Homework 1 - Esercizio 3 
## Pierluigi Compagnone

In [1]:
import scipy
from scipy import io
import cvxpy as cp
import numpy as np

In [2]:
#loading the useful files

file = io.loadmat('files/capacities.mat')
capacities = file.get('capacities')
capacities = capacities.reshape(28,)

file = io.loadmat('files/traveltime.mat')
traveltime = file.get('traveltime')
traveltime = traveltime.reshape(28,)

file = io.loadmat('files/flow.mat')
flow = file.get('flow')
flow = flow.reshape(28,)

file = io.loadmat('files/traffic.mat')
traffic = file.get('traffic')


**(a)** Find the shortest path between node 1 and 17. This is equivalent to the fastest path (path
with shortest traveling time) in an empty network

In [3]:
#finding the number of nodes and links from the node-link incidence matrix 
#and setting the exogenous net flow vector for the shortest path between 1 and 17

n_nodes = traffic.shape[0]
n_edges = traffic.shape[1]

source_node = 1
destination_node = 17

nu = np.zeros(n_nodes)
nu[source_node-1] = 1
nu[destination_node-1] = -1 

#defining the shortest path optimization problem

f_sp = cp.Variable(n_edges)
objective_sp = cp.Minimize(traveltime.T @ f_sp)
constraints_sp = [traffic @ f_sp == nu, f_sp >=0]
sp_problem = cp.Problem(objective_sp, constraints_sp)

res = sp_problem.solve()
f =  np.round(f_sp.value,2)

print("The resulting flow vector is:", f)

sp = []
for i in range(len(f)):
    if f[i] == 1:
        sp.append("l{}".format(i+1))
        
print("\nSo the shortest path is: ", sp,"\n")        

The resulting flow vector is: [1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 0.]

So the shortest path is:  ['l1', 'l2', 'l9', 'l12', 'l25'] 



**(b)** Find the maximum flow between node 1 and 17.

In [4]:
#defining and solving the maximum flow problem

tau = cp.Variable()
flows = cp.Variable(n_edges)
objective = cp.Maximize(tau)
constraints = [tau >= 0, flows >= 0, flows <= capacities, traffic @ flows == tau*nu]
p = cp.Problem(objective, constraints)

res = p.solve()

print("The maximum throughput between 1 and 17 is:", np.round(tau.value,2))
print("\nand it is obtained with the flow vector:", np.round(flows.value,2),"\n")

The maximum throughput between 1 and 17 is: 22448.0

and it is obtained with the flow vector: [ 8741.    7200.35  5181.46  3471.54 13707.    6025.3   4904.63  3485.32
  3558.71  1540.65  1010.41  1008.48  1709.92  3471.54  7681.7   4795.81
  5547.21  2661.32  2429.72  2645.01  5274.99  7920.    3317.94  1950.54
  8160.76  6789.49  7497.75  7497.75] 



**(c)** Given the flow vector in flow.mat, compute the external inflow $v$ satisfying $Bf = ν$.

In [5]:
#calculating v

v = traffic @ flow
print("the resulting external inflow is v: ", v,"\n")

the resulting external inflow is v:  [ 16806   8570  19448   4957   -746   4768    413     -2  -5671   1169
     -5  -7131   -380  -7412  -7810  -3430 -23544] 



In the following, we assume that the exogenous inflow is zero in all the nodes except for node $1$,
for which $ν_1$ has the same value computed in the point **(c)**, and node $17$, for which $ν_{17} = −ν_1$.

In [6]:
#building exogenous flow as written in the assignment

v1 = v[0]
exogenous_inflow = np.zeros(n_nodes)
exogenous_inflow[source_node-1] = v1
exogenous_inflow[destination_node-1] = -v1
print("the exogenous flow becomes: ",exogenous_inflow,"\n")

the exogenous flow becomes:  [ 16806.      0.      0.      0.      0.      0.      0.      0.      0.
      0.      0.      0.      0.      0.      0.      0. -16806.] 



**(d)** Find the social optimum $f^*$ with respect to the delays on the different links $d_e(f_e)$.

In [7]:
#defining the social optimum traffic assignment problem

flows_SO = cp.Variable(n_edges)
objective_SO = cp.Minimize((traveltime * capacities) @ cp.power((1 - flows_SO / capacities),-1) - traveltime @ capacities)
constraints_SO = [traffic @ flows_SO == exogenous_inflow, flows_SO >= 0]



problem_SO = cp.Problem(objective_SO, constraints_SO)

res_SO = problem_SO.solve()

print("The social optimum flow is: ", np.round(flows_SO.value,2))
print("\nThe cost relative to this flow is: ", np.round(res_SO,2),"\n")


The social optimum flow is:  [ 6642.3   6058.9   3132.4   3132.4  10163.7   4638.4   3006.36  2542.59
  3131.52   583.4      0.    2926.5      0.    3132.4   5525.3   2854.3
  4886.44  2215.44   463.78  2337.56  3318.08  5655.64  2373.04     0.
  6414.12  5505.44  4886.44  4886.44]

The cost relative to this flow is:  25943.62 



**(e)** Find the Wardrop equilibrium $f^{(0)}$

In [8]:
#calculate the Wardrop equilibrium flow

flows_W = cp.Variable(n_edges)
objective_W = cp.Minimize((traveltime * capacities) @ (np.log(capacities) - cp.log(capacities-flows_W)))

constraints_W = [traffic @ flows_W == exogenous_inflow, flows_W >= 0]

problem_W = cp.Problem(objective_W, constraints_W)

res_W = problem_W.solve()

print("The Wardrop equilibrium flow is: ", np.round(flows_W.value,2))
print("\nThe cost relative to this flow is: ", np.round(res_W,2),"\n")

The Wardrop equilibrium flow is:  [ 6715.65  6715.64  2367.41  2367.41 10090.35  4645.39  2803.84  2283.56
  3418.48     0.     176.83  4171.41     0.    2367.41  5444.96  2353.17
  4933.34  1841.55   697.11  3036.49  3050.28  6086.77  2586.51     0.
  6918.74  4953.92  4933.34  4933.34]

The cost relative to this flow is:  15729.61 



In [9]:
#calculating the price of anarchy to understan how much worse is the Wardrop equilibrium than Social optimum

#total delay function

def so_cost(f):
    cost_vec = (traveltime * capacities) / (1 - f / capacities) - traveltime * capacities
    return cost_vec.sum()

#calculate the total delay of the Wardrop equilibrium

cost_w = so_cost(flows_W.value)

print("the total delay at the Wardrop equilibrium is: ",np.round(cost_w,2),"\n")

PoA = cost_w/res_SO
print("the price of anarchy is: ", np.round(PoA,4),"\n")


the total delay at the Wardrop equilibrium is:  26292.96 

the price of anarchy is:  1.0135 



**(e)** Introduce tolls, such that the toll on link $e$ is $ω_e = f_e^* d_e'(f_e^* )$, where $f_e^* $ is the flow at the
system optimum. Now the delay on link e is given by $d_e(f_e) + ω_e$. compute the new Wardrop
equilibrium $f^{(ω)}$. What do you observe?

In [10]:
#calculating the tolls

SO_flow = flows_SO.value
w = SO_flow * ((traveltime * capacities)/(capacities - SO_flow)**2) 
print("The constructed tolls are: ", np.round(w,2))

The constructed tolls are:  [1.92 0.19 0.05 0.11 1.44 0.47 0.11 0.06 0.28 0.01 0.   0.08 0.   0.13
 0.48 0.08 0.07 0.02 0.   0.01 0.07 0.26 0.07 0.   0.41 0.29 0.19 0.53]


In [11]:
#defining the Wardrop equilibrium flow with tolls

flows_W1 = cp.Variable(n_edges)
objective_W1 = cp.Minimize(((traveltime * capacities) @ (np.log(capacities) - cp.log(capacities-flows_W1))) + w @ flows_W1)
constraints_W1 = [traffic @ flows_W1 == exogenous_inflow, flows_W1 >= 0]

problem_W1 = cp.Problem(objective_W1, constraints_W1)

res_W1 = problem_W1.solve()

print("The new Wardrop equilibrium flow is: ", np.round(flows_W1.value,2))
print("\nThe cost relative to this flow is: ", np.round(res_W1,2))

The new Wardrop equilibrium flow is:  [ 6642.3   6059.07  3132.3   3132.3  10163.69  4638.01  3006.25  2542.45
  3131.54   583.24     0.    2926.77     0.    3132.3   5525.68  2854.25
  4886.43  2215.     463.8   2337.68  3318.06  5655.73  2373.17     0.
  6414.11  5505.46  4886.43  4886.43]

The cost relative to this flow is:  61885.97


In [12]:
#calculating the price of anarchy of the Wardrop equilibrium with tolls

#calculating the total delay of the Wardrop equilibrium with tolls 

cost_w1 = so_cost(flows_W1.value)

print("the total delay at the Wardrop equilibrium with tolls is: ",np.round(cost_w1,2),"\n")

PoA1 = cost_w1/res_SO
print("\nthe price of anarchy after the introduction of tolls is: ", np.round(PoA1,4),"\n")

if np.round(PoA1) == 1:
    print("So the tolls are well designed\n")

the total delay at the Wardrop equilibrium with tolls is:  25943.62 


the price of anarchy after the introduction of tolls is:  1.0 

So the tolls are well designed



**(d)** Instead of the total delay, let the cost be the total additional delay compared to the total delay in free flow.

$$c_e(f_e) = f_e(d_e(f_e) −l_e)$$

subject to the flow constraints. Compute the system optimum $f^∗$ for the costs above. 
Construct tolls $ω_e^*$, $e ∈ \epsilon$ such that the new Wardrop equilibrium with the constructed tolls $f(ω^∗)$
coincides with $f^∗$. Compute the new Wardrop equilibrium with the constructed tolls $f(ω^∗)$
to verify your result.

In [13]:
#calculate the social optimum flow using the new delay defined in point (d)

flows_SO1 = cp.Variable(n_edges)
objective_SO1 = cp.Minimize(((traveltime * capacities) @ cp.power((1 - flows_SO1 / capacities),-1) - traveltime @ capacities) - traveltime @ flows_SO1)
constraints_SO1 = [traffic @ flows_SO1 == exogenous_inflow, flows_SO1 >= 0]



problem_SO1 = cp.Problem(objective_SO1, constraints_SO1)

res_SO1 = problem_SO1.solve()

print("The new social optimum flow is: ", np.round(flows_SO1.value,1) )

print("\nThe cost relative to this flow is: ", np.round(res_SO1,2),"\n")


The new social optimum flow is:  [ 6653.3  5774.7  3419.7  3419.7 10152.7  4642.7  3105.8  2662.2  3009.1
   878.6     0.   2354.9     0.   3419.7  5510.   3043.7  4881.8  2415.5
   443.7  2008.   3487.4  5495.4  2203.8     0.   6300.7  5623.5  4881.8
  4881.8]

The cost relative to this flow is:  15095.51 



In [14]:
#calculate the new tolls

SO1_flow = flows_SO1.value
w1 = SO1_flow * ((traveltime * capacities)/(capacities - SO1_flow)**2)
print("the tolls to introduce are: ", np.round(w1,2),"\n")

the tolls to introduce are:  [1.95 0.15 0.06 0.12 1.43 0.47 0.12 0.06 0.25 0.01 0.   0.05 0.   0.15
 0.48 0.1  0.07 0.02 0.   0.01 0.07 0.24 0.06 0.   0.38 0.31 0.19 0.53] 



In [15]:
#calculate the Wardrop equilibrium flow considering the new additional delay function and the tolls calculated in the previous step

flows_W2 = cp.Variable(n_edges)
objective_W2 = cp.Minimize(((traveltime * capacities) @ (np.log(capacities) - cp.log(capacities-flows_W2))) - (traveltime @ flows_W2) + (w1 @ flows_W2))
constraints_W2 = [traffic @ flows_W2 == exogenous_inflow, flows_W2 >= 0]

problem_W2 = cp.Problem(objective_W2, constraints_W2)

res_W2 = problem_W2.solve()

print("The Wardrop equilibrium flow, considering the tolls and the new delay function, is: ", np.round(flows_W2.value,2))
print("\nThe cost relative to this flow is: ", np.round(res_W2,2),"\n")

The Wardrop equilibrium flow, considering the tolls and the new delay function, is:  [ 6653.37  5775.47  3419.42  3419.42 10152.63  4642.73  3105.49  2661.73
  3009.22   877.9      0.    2356.05     0.    3419.42  5509.9   3043.32
  4881.71  2415.14   443.77  2008.56  3487.09  5495.65  2204.03     0.
  6300.84  5623.45  4881.71  4881.71]

The cost relative to this flow is:  50795.81 



In [16]:
#calculate the price of anarchy to check if the tolls are well designed

#new total delay function

def so1_cost(f):
    cost_vec = ((traveltime * capacities)/(1 - f / capacities) - traveltime * capacities) - traveltime * f
    return cost_vec.sum()

#total delay for founded Wardrop equilibrium with tolls

cost_w2 = so1_cost(flows_W2.value)
print("The total delay at the Wardrop equilibrium with the constructed tolls is: ", np.round(cost_w1,2))

PoA2 = cost_w2/res_SO1

print("\nThe correspondent price of anarchy is: ", np.round(PoA2))

if np.round(PoA2) == 1:
    print("\nSo the tolls are well designed\n")

The total delay at the Wardrop equilibrium with the constructed tolls is:  25943.62

The correspondent price of anarchy is:  1.0

So the tolls are well designed

